In [20]:
class YOutputRescaler(object):
    def __init__(self,ANCHORS):
        self.ANCHORS = ANCHORS

    def _sigmoid(self, x):
        return 1. / (1. + np.exp(-x))
    def _softmax(self, x, axis=-1, t=-100.):
        x = x - np.max(x)

        if np.min(x) < t:
            x = x/np.min(x)*t

        e_x = np.exp(x)
        return e_x / e_x.sum(axis, keepdims=True)
    def get_shifting_matrix(self,netout):
                
        GRID_H, GRID_W, BOX = netout.shape[:3]
        no = netout[...,0]
        
        ANCHORSw = self.ANCHORS[::2]
        ANCHORSh = self.ANCHORS[1::2]
       
        mat_GRID_W = np.zeros_like(no)
        for igrid_w in range(GRID_W):
            mat_GRID_W[:,igrid_w,:] = igrid_w

        mat_GRID_H = np.zeros_like(no)
        for igrid_h in range(GRID_H):
            mat_GRID_H[igrid_h,:,:] = igrid_h

        mat_ANCHOR_W = np.zeros_like(no)
        for ianchor in range(BOX):    
            mat_ANCHOR_W[:,:,ianchor] = ANCHORSw[ianchor]

        mat_ANCHOR_H = np.zeros_like(no) 
        for ianchor in range(BOX):    
            mat_ANCHOR_H[:,:,ianchor] = ANCHORSh[ianchor]
        return(mat_GRID_W,mat_GRID_H,mat_ANCHOR_W,mat_ANCHOR_H)

    def fit(self, netout_to_fit):    
        '''
        netout  : np.array of shape (N grid h, N grid w, N anchor, 4 + 1 + N class)
        
        a single image output of model.predict()
        '''
        netout = copy.deepcopy(netout_to_fit)
        
        GRID_H, GRID_W, BOX = netout.shape[:3]
        
        (mat_GRID_W,
         mat_GRID_H,
         mat_ANCHOR_W,
         mat_ANCHOR_H) = self.get_shifting_matrix(netout)


         # bounding box parameters
        netout[..., 0]   = (self._sigmoid(netout[..., 0]) + mat_GRID_W)#/GRID_W # x      unit: range between 0 and 1
        netout[..., 1]   = (self._sigmoid(netout[..., 1]) + mat_GRID_H)#/GRID_H # y      unit: range between 0 and 1
        netout[..., 2]   = (np.exp(netout[..., 2]) * mat_ANCHOR_W)#/GRID_W      # width  unit: range between 0 and 1
        netout[..., 3]   = (np.exp(netout[..., 3]) * mat_ANCHOR_H)#/GRID_H      # height unit: range between 0 and 1
        # rescale the confidence to range 0 and 1 
        netout[..., 4]   = self._sigmoid(netout[..., 4])
        expand_conf      = np.expand_dims(netout[...,4],-1) # (N grid h , N grid w, N anchor , 1)
        # rescale the class probability to range between 0 and 1
        # Pr(object class = k) = Pr(object exists) * Pr(object class = k |object exists)
        #                      = Conf * P^c
        netout[..., 5:]  = expand_conf * self._softmax(netout[..., 5:])
        # ignore the class probability if it is less than obj_threshold 

        return(netout)

    

    

In [21]:

# class BestAnchorBoxFinder3(object):
    
#     def __init__(self, box_true,box_true_list, ANCHORS):
#         '''
#         ANCHORS: a np.array of even number length e.g.
        
#         _ANCHORS = [4,2, ##  width=4, height=2,  flat large anchor box
#                     2,4, ##  width=2, height=4,  tall large anchor box
#                     1,1] ##  width=1, height=1,  small anchor box
#         '''
#         self.anchors = [BoundBox(0, 0, ANCHORS[2*i], ANCHORS[2*i+1]) 
#                         for i in range(int(len(ANCHORS)//2))]
        
#         self.box_true = box_true
#         self.box_true_list = box_true_list
        
#     def _interval_overlap(self,interval_a, interval_b):
#         x1, x2 = interval_a
#         x3, x4 = interval_b
#         if x3 < x1:
#             if x4 < x1:
#                 return 0
#             else:
#                 return min(x2,x4) - x1
#         else:
#             if x2 < x3:
#                  return 0
#             else:
#                 return min(x2,x4) - x3  

#     def bbox_iou(self,box1, box2):
#         print('[box1.xmin, box1.xmax]',[box1.xmin, box1.xmax])
#         print('[box2.xmin, box2.xmax]',[box2.xmin, box2.xmax])
#         print('[box1.ymin, box1.ymax]',[box1.ymin, box1.ymax])
#         print('[box2.ymin, box2.ymax]',[box2.ymin, box2.ymax])        
        
        
#         if box1.xmin < 0:
#             box1.xmin = 0
#         if box1.ymin < 0:
#             box1.ymin = 0            
#         if box1.xmax < 0:
#             box1.xmax = 0            
#         if box1.ymax < 0:
#             box1.ymax = 0            
        
#         print('Adjusted Boxes')
#         print('[box1.xmin, box1.xmax]',[box1.xmin, box1.xmax])
#         print('[box2.xmin, box2.xmax]',[box2.xmin, box2.xmax])
#         print('[box1.ymin, box1.ymax]',[box1.ymin, box1.ymax])
#         print('[box2.ymin, box2.ymax]',[box2.ymin, box2.ymax])   
        
        
#         intersect_w = self._interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
#         intersect_h = self._interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])  

#         intersect = intersect_w * intersect_h
#         print('intersect',intersect)
#         w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
#         w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin

#         union = w1*h1 + w2*h2 - intersect
#         print('union', union)
#         return float(intersect) / union
    

        
#     def find2(self,box_test, label_predicted = 0, labels = 0):
#         # find the anchor that best predicts this box
#         best_anchor = -1
#         max_iou     = -1
#         match = 0
#         match_box = []
#         best_box = -1

#         for i in range(len(self.box_true_list)): ## run through each anchor box
#             box_t = BoundBox(self.box_true_list[i][0][0],self.box_true_list[i][0][1],self.box_true_list[i][0][2],self.box_true_list[i][0][3],1)
    
#             label_t = self.box_true_list[i][1]
#             box_test.label = label_predicted 
#             print('box_test.label   ',box_test.label)
#             print('label_predicted: ',label_predicted)
#             print('label_t ',label_t)
#             if (label_predicted == label_t):
#                 print('match: box_t', box_t)
#             iou    = self.bbox_iou(box_test, box_t)
    
#             print('iou is: ',iou)
#             if max_iou < iou:
#                 match = 1
#                 if (label_predicted == label_t):
#                     best_box = i
#                     max_iou     = iou
                    
                    
#         if match == 1 and best_box != -1:
#             match_box = self.box_true_list.pop(best_box)
#             print('box_true_list len ',len(self.box_true_list))
#             print('box test is: ',box_test)
        
                
#         return(match,match_box, best_box, max_iou,self.box_true_list)    


      

# def find_high_class_probability_bbox2_backup(netout_scale2,obj_threshold, test_image,boxes,image,LABELS,ious_threshold):
#     '''
#     == Input == 
#     netout : y_pred[i] np.array of shape (GRID_H, GRID_W, BOX, 4 + 1 + N class)
    
#              x, w must be a unit of image width
#              y, h must be a unit of image height
#              c must be in between 0 and 1
#              p^c must be in between 0 and 1
#     == Output ==
    
#     boxes  : list containing bounding box with Pr(object is in class C) > 0 for at least in one class C 
    
             
#     '''
    
#     netout_scale = netout_scale2
#     print('netout_scale shape: ', netout_scale.shape)
    
#     GRID_H, GRID_W, BOX = netout_scale.shape[:3]

#     def adjust_minmax(c,_max):
#         if c < 0:
#             c = 0   
#         if c > _max:
#             c = _max
#         return c
               
    
#     labels = LABELS
    
#     Box_true_list = []
#     Box_true = []
   
    
#     for i in range(len(test_image["object"])):
#         xmin_true = test_image["object"][i]["xmin"]    
#         xmax_true = test_image["object"][i]["xmax"] 
#         ymin_true = test_image["object"][i]["ymin"] 
#         ymax_true = test_image["object"][i]["ymax"] 

#         temp = [xmin_true, ymin_true, xmax_true, ymax_true]
#         print('True: xmin, ymin, xmax, ymax ',[xmin_true, ymin_true, xmax_true, ymax_true])
        
#         y, h, w, c = image.shape
#         print('DEBUG size: ', w,h)
#         IMAGE_W = 416
#         IMAGE_H = 416

#  #       print('After Adjustment : ')
#         xmin_try = int(xmin_true * float(IMAGE_W) / w)
#         xmin_try = max(min(xmin_true, IMAGE_W), 0)
#         xmax_try = int(xmax_true * float(IMAGE_W) / w)
#         xmax_try = max(min(xmax_true, IMAGE_W), 0)

#         ymin_try = int(ymin_true * float(IMAGE_H) / h)
#         ymin_try = max(min(ymin_true, IMAGE_H), 0)
#         ymax_try = int(ymax_true * float(IMAGE_H) / h)
#         ymax_try = max(min(ymax_true, IMAGE_H), 0)        
     
#         print('True: xmin, ymin, xmax, ymax ',[xmin_try, ymin_try, xmax_try, ymax_try])
#         temp = [xmin_try, ymin_try, xmax_try, ymax_try]

#         label_true = test_image["object"][0]["name"]
        
#         Box_true.append(temp)
#         Box_true_list.append([temp, label_true])#classes_true])
    
#     count_total = 0
#     FP = 0
#     TP = 0
    
#     obj_baseline=0.05
    
#     image = copy.deepcopy(image)
#     _, image_h, image_w, _ = image.shape
#     score_rescaled  = np.array([box.get_score() for box in boxes])
#     score_rescaled /= obj_baseline
   
#     for sr, box in zip(score_rescaled,boxes):
#         xmin = adjust_minmax(int(box.xmin*image_w),image_w)
#         ymin = adjust_minmax(int(box.ymin*image_h),image_h)
#         xmax = adjust_minmax(int(box.xmax*image_w),image_w)
#         ymax = adjust_minmax(int(box.ymax*image_h),image_h)
        
   
    
#         label_predicted = labels[box.label]
#         print('label_predicted: ', label_predicted)
#         print('Predicted: xmin, ymin, xmax, ymax ',[xmin, ymin, xmax, ymax])
        
#         box = BoundBox(xmin,ymin, xmax, ymax, box.get_score())#, classes)

#         compare = BestAnchorBoxFinder2(Box_true,Box_true_list, ANCHORS)
#         match,match_box, best_box, max_iou,Box_true_list = compare.find2(box,label_predicted, labels)      
        
#         print('first box')
#         print('match: ', match)
#         print('best_box: ', best_box)
    
#         if max_iou > ious_threshold:

#             if match == 1:
#                 print('match')
#                 count_total+=1
#                 TP+=1
#             else:
#                 count_total+=1
#                 FP+=1
#                 print(result)

#     percision = 0                        
#     print('count_total = ',count_total)
#     print('TP = ', TP)
#     print('FP = ', FP)
#     if count_total > 0:
#         percision = float(TP/count_total)
#         print('pecision = ', percision)

#     return percision




In [22]:

class BestAnchorBoxFinder2(object):
    
    def __init__(self, box_true,predicted_boxes_list, ANCHORS):
        '''
        ANCHORS: a np.array of even number length e.g.
        
        _ANCHORS = [4,2, ##  width=4, height=2,  flat large anchor box
                    2,4, ##  width=2, height=4,  tall large anchor box
                    1,1] ##  width=1, height=1,  small anchor box
        '''
        self.anchors = [BoundBox(0, 0, ANCHORS[2*i], ANCHORS[2*i+1]) 
                        for i in range(int(len(ANCHORS)//2))]
        
        self.box_true = box_true
        self.predicted_boxes_list = predicted_boxes_list
        
    def _interval_overlap(self,interval_a, interval_b):
        x1, x2 = interval_a
        x3, x4 = interval_b
        if x3 < x1:
            if x4 < x1:
                return 0
            else:
                return min(x2,x4) - x1
        else:
            if x2 < x3:
                 return 0
            else:
                return min(x2,x4) - x3  

    def bbox_iou(self,box1, box2):
#         print('[box1.xmin, box1.xmax]',[box1.xmin, box1.xmax])
#         print('[box2.xmin, box2.xmax]',[box2.xmin, box2.xmax])
#         print('[box1.ymin, box1.ymax]',[box1.ymin, box1.ymax])
#         print('[box2.ymin, box2.ymax]',[box2.ymin, box2.ymax])        
        
        
        if box1.xmin < 0:
            box1.xmin = 0
        if box1.ymin < 0:
            box1.ymin = 0            
        if box1.xmax < 0:
            box1.xmax = 0            
        if box1.ymax < 0:
            box1.ymax = 0            
 
        
        intersect_w = self._interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
        intersect_h = self._interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])  

        intersect = intersect_w * intersect_h
        w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
        w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin

        union = w1*h1 + w2*h2 - intersect
        return float(intersect) / union
    

        
    def find2(self,box_true, label_true = 0, labels = 0):
        # find the anchor that best predicts this box
        best_anchor = -1
        max_iou     = -1
        match = 0
        match_box = []
        best_box = -1

        for i in range(len(self.predicted_boxes_list)): ## run through each anchor box
    
            label_pred = self.predicted_boxes_list[i][1]
            box_true.label = label_true 

            box_pred = BoundBox(self.predicted_boxes_list[i][0].xmin,self.predicted_boxes_list[i][0].ymin,self.predicted_boxes_list[i][0].xmax,self.predicted_boxes_list[i][0].ymax,self.predicted_boxes_list[i][1])

            iou    = self.bbox_iou(box_true, box_pred)
    
            if max_iou < iou:
                if (label_true == label_pred):
                    match = 1
                    best_box = i
                    max_iou     = iou
                    
                    
        if match == 1 and best_box != -1:
            match_box = self.predicted_boxes_list.pop(best_box)
        
        return(match,match_box, best_box, max_iou,self.predicted_boxes_list)    


      
def find_high_class_probability_bbox2(TP_list,FP_list,FN_list, netout_scale2 ,obj_threshold, test_image,boxes,image,LABELS,ious_threshold):
    '''
    == Input == 
    netout : y_pred[i] np.array of shape (GRID_H, GRID_W, BOX, 4 + 1 + N class)
    
             x, w must be a unit of image width
             y, h must be a unit of image height
             c must be in between 0 and 1
             p^c must be in between 0 and 1
    == Output ==
    
    boxes  : list containing bounding box with Pr(object is in class C) > 0 for at least in one class C 
    
             
    '''
    
    netout_scale = netout_scale2
#    print('netout_scale shape: ', netout_scale.shape)
    
    GRID_H, GRID_W, BOX = netout_scale.shape[:3]

    def adjust_minmax(c,_max):
        if c < 0:
            c = 0   
        if c > _max:
            c = _max
        return c
               
    
    labels = LABELS
    
    Box_true_list = []
    Box_true = []
   
    
    for i in range(len(test_image["object"])):
        xmin_true = test_image["object"][i]["xmin"]    
        xmax_true = test_image["object"][i]["xmax"] 
        ymin_true = test_image["object"][i]["ymin"] 
        ymax_true = test_image["object"][i]["ymax"] 
#        if i ==0:
#            print('example: ', test_image)

        temp = [xmin_true, ymin_true, xmax_true, ymax_true]
#        print('True: xmin, ymin, xmax, ymax ',[xmin_true, ymin_true, xmax_true, ymax_true])
#        print('image.shape: ', image.shape)
        try:
            u, h, w, c = image.shape
        except:
            h, w, c = image.shape

        IMAGE_W = 416
        IMAGE_H = 416

 #       print('After Adjustment : ')
        xmin_try = int(xmin_true * float(IMAGE_W) / w)
        xmin_try = max(min(xmin_true, IMAGE_W), 0)
        xmax_try = int(xmax_true * float(IMAGE_W) / w)
        xmax_try = max(min(xmax_true, IMAGE_W), 0)

        ymin_try = int(ymin_true * float(IMAGE_H) / h)
        ymin_try = max(min(ymin_true, IMAGE_H), 0)
        ymax_try = int(ymax_true * float(IMAGE_H) / h)
        ymax_try = max(min(ymax_true, IMAGE_H), 0)        
     
#        print('True: xmin, ymin, xmax, ymax ',[xmin_try, ymin_try, xmax_try, ymax_try])

        label_true = test_image["object"][0]["name"]
        
        Box_true.append(temp)
        Box_true_list.append([temp, label_true])
    
    count_total = 0
    FP = 0
    TP = 0
    
    obj_baseline=0.05
    
    image = copy.deepcopy(image)
    try:
        _, image_h, image_w, _ = image.shape
    except:
        image_h, image_w, _ = image.shape
    score_rescaled  = np.array([box.get_score() for box in boxes])
    score_rescaled /= obj_baseline

    predicted_boxes_list = []
    
    for sr, box in zip(score_rescaled,boxes):

        if (box.xmin in [-np.inf,np.inf]) or (box.xmax in [-np.inf,np.inf]) or (box.ymin in [-np.inf,np.inf]) or (box.ymax in [-np.inf,np.inf]):
            continue
        xmin = adjust_minmax(int(box.xmin*image_w),image_w)
        ymin = adjust_minmax(int(box.ymin*image_h),image_h)
        xmax = adjust_minmax(int(box.xmax*image_w),image_w)
        ymax = adjust_minmax(int(box.ymax*image_h),image_h)
        
    
        label_predicted = labels[box.label]

        if box.get_score() > obj_threshold:
            box = BoundBox(xmin,ymin, xmax, ymax, box.get_score())#, classes)
        
            predicted_boxes_list.append([box, label_predicted])
        
        
    for box_true, label_true in Box_true_list:
        box_true = BoundBox(box_true[0],box_true[1],box_true[2],box_true[3],label_true)
        compare = BestAnchorBoxFinder2(Box_true,predicted_boxes_list, ANCHORS)
        match,match_box, best_box, max_iou,predicted_boxes_list = compare.find2(box_true,label_true, labels)      
        

        if max_iou > ious_threshold:

            if match == 1:
                count_total+=1
                TP+=1
                TP_list[labels.index(label_true)]+=1
            else:
                count_total+=1
                FP+=1
                FN_list[labels.index(label_true)]+=1
        else:
            FN_list[labels.index(label_true)]+=1

    FP = len(predicted_boxes_list)

    # For each unmatched class add 
    for unpredicted_box in predicted_boxes_list:
        FP_list[labels.index(unpredicted_box[1])]+=1

    percision = 0                 

    if count_total > 0:
        percision = float(TP/(TP+FP))
    return percision, TP_list,FP_list, FN_list